# Lab work 2
## Numeric solution of a linear equations system

## 1. Gauss elimination algorithm

It is a simple algorithm:  
a) firstly we exclude variable with using elementary lines transformations, making a upper triangular matrix  
b) then we sequentially find all variable since the last

In [9]:
import numpy as np
import numpy.linalg as la

def dummy_Gauss(A, b):
    n = b.size
    
    # forward elimination
    for j in range(0, n-1):
        for i in range(j+1, n):
            if A[j,j] != 0:
                num = A[i,j] / A[j,j]
                A[i,j:n] -= num*A[j,j:n]
                b[i] -= num*b[j]
            
    # back substitution
    res = np.zeros(n) 
    for i in range(n-1, -1, -1):
        if A[i,i] != 0:
            res[i] = (b[i] - np.dot(A[i,i+1:n], res[i+1:n])) / A[i,i]
    return res
    
# test
A1 = np.array([[-3., -1.,  2.], 
              [ 2.,  1., -1.], 
              [-2.,  1.,  2.]])
b1 = np.array([-11.,8.,-3.])
print('mu1 = ', la.cond(A1))
print("Embedded algorithm: ", la.solve(A1, b1))
print("Gauss algorithm: ", dummy_Gauss(A1, b1))
print("\n")

# this system will be solved incorrectly
# because of dividing by a big number
A2 = np.array([[1e-16,  1., -1.], 
               [  -1.,  2., -1.], 
               [   2., -1.,  0.]]);
b2 = np.array([0., 0., 1.])
print('mu2 = ', la.cond(A2))
print("Embedded algorithm: ", la.solve(A2, b2))
print("Gauss algorithm: ", dummy_Gauss(A2, b2))

mu1 =  44.672322469556505
Embedded algorithm:  [ 2.  3. -1.]
Gauss algorithm:  [ 2.  3. -1.]


mu2 =  16.393731622284385
Embedded algorithm:  [1. 1. 1.]
Gauss algorithm:  [0.55511151 0.25       0.25      ]


That algorithm was bad because of big inaccuracy, appearing when the diagonal element is sufficient smaller than others.

The complexity is $O\left(\frac{2}{3}n^3\right)$

There was used the conditional number counting as $\mu = \left\|A^{-1}\right\| \cdot \|A\| $ that measuring how much results can change because of small difference in input data.

## 1.1 Gauss elimination algorithm with the selection of the main element

Let's improve Gauss algorithm. It had a big inaccuracy, when the diagonal element is sufficient smaller than others.  
Now let's change lines to find the maximal element and divide by it.  
It will reduce inaccuracy.

In [10]:
def smart_Gauss(A, b):
    n = b.size
    
    # forward elimination
    for j in range(0, n-1):
        for i in range(j+1, n):
            
            #find a max element in j column
            max = abs(A[j,j])
            str = j
            for k in range(j+1, n):
                if abs(A[k,j]) > max:
                    max = abs(A[k,j])
                    str = k
            
            # if max element is 0, matrix is singular
            if max == 0:
                raise la.LinAlgError("Singular matrix")
                
            # else change rows' positions
            else:
                for k in range(n):
                    tmp = A[j,k]
                    A[j,k] = A[str,k]
                    A[str,k] = tmp
            b[j], b[str] = b[str], b[j]
            
            num = A[i,j] / A[j,j]
            A[i,j:n] -= num*A[j,j:n]
            b[i] -= num*b[j]
                
    # back substitution
    res = np.zeros(n)
    # necessary and sufficient conditions of convergence of Gauss method
    for i in range(n-1, -1, -1):
        if A[i,i] != 0:
            res[i] = (b[i] - np.dot(A[i,i+1:n], res[i+1:n])) / A[i,i]
        else:
            raise la.LinAlgError("Singular matrix")
    return res

# test
try:
    A1 = np.array([[1e-16,  1., -1.], 
                   [  -1.,  2., -1.], 
                   [   2., -1.,  0.]]);
    b1 = np.array([0., 0., 1.])
    print('mu1 = ', la.cond(A1))
    print("Gauss algorithm: ", smart_Gauss(A1, b1))
    print("Embedded algorithm: ", la.solve(A1, b1))
    print("\n")
    
except la.LinAlgError as lae:
    print("LinAlgError exception: " + str(lae))
    
try:
    A2 = np.array([[100.,99.], 
                   [ 99.,98.]]);
    b2 = np.array([199., 197.])
    A21 = np.array([[100.,99.], 
                    [ 99.,98.]]);
    b21 = np.array([198.99, 197.01])
    print('mu2 = ', la.cond(A2))
    print("Gauss algorithm: ", smart_Gauss(A2, b2))
    print("Gauss algorithm for changed: ", smart_Gauss(A21, b21))
    print("\n")
    
except la.LinAlgError as lae:
    print("LinAlgError exception: " + str(lae))

try:
    A3 = np.array([[1., 2., 3.], 
                   [2., 4., 6.], 
                   [0., 1., 2.]])
    b3 = np.array([8., -11., -3.])
    print('mu3 = ', la.cond(A3))
    # there will be an exception
    print("Gauss algorithm: ", smart_Gauss(A3, b3))
    print("\n")
    
except la.LinAlgError as lae:
    print("LinAlgError exception: " + str(lae))
    

mu1 =  16.393731622284385
Gauss algorithm:  [1. 1. 1.]
Embedded algorithm:  [1. 1. 1.]


mu2 =  39205.999974425096
Gauss algorithm:  [1. 1.]
Gauss algorithm for changed:  [ 2.97 -0.99]


mu3 =  2.453817034094917e+16
LinAlgError exception: Singular matrix


In example 1, how one can see, this algorithm is better - it gives correct answer.  
Example 2 shows, how small difference (about 0.01) can change the result if the matrix has bad condition number: 
$\mu \approx 4\cdot10^4$  
And the example 3 shows that system with singular matrix cannot be solved.

## 2. Tridiagonal matrix algorithm or Thomas algorithm

This algorithm solve a system $ Ax = b $ with 3-diagonal matrix:
$$
A = \begin{pmatrix} 
\beta_1 & \gamma_1 \\
\alpha_2 & \beta_2 & \gamma_2 \\
& \ddots & \ddots & \ddots \\
& & \alpha_{n-1} & \beta_{n-1} & \gamma_{n-1} \\
& & & \alpha_n & \beta_n \\
\end{pmatrix}
$$
Assuming $$ x_i = P_i x_{i+1} + Q_i, ~~ x_n = Q_n$$
where coefficients can be calculated as 
$$ P_i = \frac{\gamma_i}{-\beta_i - \alpha_i P_{i-1}} \\
   Q_i = \frac{\alpha_i Q_{i-1} - b_i}{-\beta_i - \alpha_i P_{i-1}} \\
   P_1 = \frac{\gamma_1}{-\beta_1}, ~~ Q_1 = \frac{b_1}{\beta_1}, ~~ P_n = 0 $$
   
The complexity is $ O(n) $

In [11]:
def checkDominance(a):
    n = len(a)
    
    # sufficient condition: diagonally dominant matrix
    if abs(a[0,0]) < abs(a[0,1]):
        print("no diagonally dominant")
        return
    for i in range(1, n-1):
        if abs(a[i,i]) < abs(a[i,i-1]) + abs(a[i,i+1]):
            print("no diagonally dominant")
            return
    if abs(a[n-1,n-1]) < abs(a[n-1,n-2]):
        print("no diagonally dominant")
        return
    
    print("diagonally dominant")
    return

def tridiag(A, b):
    checkDominance(A)
        
    n = b.size
    alpha = np.diagonal(A, offset = -1)
    beta = np.diagonal(A)
    gamma = np.diagonal(A, offset = 1)
    
    P = np.zeros(n)
    P[0] = -gamma[0] / beta[0]
    for i in range(1, n-1):
        P[i] = gamma[i] / (-beta[i] - alpha[i-1]*P[i-1])
    P[n-1] = 0
    
    Q = np.zeros(n)
    Q[0] = b[0] / beta[0]
    for i in range(1, n):
        Q[i] = (alpha[i-1]*Q[i-1] - b[i]) / (-beta[i] - alpha[i-1]*P[i-1])
        
    x = np.zeros(n)
    x[n-1] = Q[n-1]
    for i in range(n-2, -1, -1):
        x[i] = P[i]*x[i+1] + Q[i]
    
    return x

# test
A1 = np.array([[5., 3., 0., 0.], 
               [3., 6., 1., 0.], 
               [0., 1., 4., -2.],
               [0., 0., 1., -3.]]);
b1 = np.array([8., 10., 3., -2.])
print("Embedded algorithm: ", la.solve(A1, b1))
print("Tridiagonal algorithm: ", tridiag(A1, b1))
print("\n")

A2 = np.array([[1.,  2.,  0., 0.], 
               [2., -1.,  1., 0.], 
               [0.,  1., -1., 1.],
               [0.,  0.,  1., 1.]]);
b2 = np.array([5., 3., 3., 7.])
print("Embedded algorithm: ", la.solve(A2, b2))
print("Tridiagonal algorithm: ", tridiag(A2, b2))

Embedded algorithm:  [1. 1. 1. 1.]
diagonally dominant
Tridiagonal algorithm:  [1. 1. 1. 1.]


Embedded algorithm:  [1. 2. 3. 4.]
no diagonally dominant
Tridiagonal algorithm:  [1. 2. 3. 4.]


In example 1, how one can see, we have diagonally dominant matrix, and everything is good.

Example 2 shows that system with matrix within diagonally dominant can also be solved with using this algorithm.
It means that sufficent condition isn't necessary

# Now let's consider iterative methods. 

## 3. Jacobi method

Let's decompose matrix $ A = L + D + U $
to diagonal component D, a lower triangular part L and an upper triangular part U  
Then make an iterative procedure $ x_{k+1} = -D^{-1}(L + U) x_k + D^{-1} b = B x_k + F $  

It will converge, if:  
a) matrix A is diagonally dominant: $|a_{ii}| > \sum\limits_{j \ne i}^{n}|a_{ij}| $  
b) $ \|B\| \le q < 1 $  
с) $ |\lambda| < 1 $ for the roots of the equation $ det(L + \lambda D + U) = 0 $  

To get accuracy = $ \varepsilon $, you should make iterations while $ \|x_{k+1} - x_{k}\| \le \frac{1-q}{q}\varepsilon $

In [12]:
def Jacobi(A, b, eps):
    checkDominance(A)
    n = b.size
    L = np.zeros((n,n))
    D = np.zeros((n,n))
    U = np.zeros((n,n))
    
    for i in range(n):
        for j in range(i):
            L[i,j] = A[i,j]
        D[i,i] = A[i,i]
        for j in range(i+1, n):
            U[i,j] = A[i,j]
    
    B = -la.inv(D).dot((L + U))
    f = la.inv(D).dot(b)
    x = np.zeros(n)
    x_new = np.zeros(n)
    q = la.norm(B, 1)
    if q >= 1:
        print("not converge")
        return x
    
    while True:
        x_new = B.dot(x) + f
        if la.norm(x_new - x, 2) <= (1-q)/q*eps:
            break
        x = x_new
        
    return x_new
    
    
# test
A1 = np.array([[ 1., -0.3,  0.2], 
               [0.3,  -1., -0.2], 
               [0.2, -0.3,   1.]])
b1 = np.array([1., -2.3, 2.6])
print("Embedded algorithm: ", la.solve(A1, b1))
print("Jacobi algorithm: ", Jacobi(A1, b1, 1e-5))
    

Embedded algorithm:  [1. 2. 3.]
diagonally dominant
Jacobi algorithm:  [1.00000041 1.99999988 3.00000041]


## 4. Gauss-Seidel method

This method is similar to Jacobi method. We decompose $ A = L + D + U $ in the same way,  
but iterative procedure differs:  $ x_{k+1} = -(L + D)^{-1}U x_k + (L + D)^{-1} b = B x_k + F $  

It will converge, if:  
a) matrix A is diagonally dominant: $|a_{ii}| > \sum\limits_{j \ne i}^{n}|a_{ij}| $  
b) $ \|B\| \le q < 1 $  
с) $ |\lambda| < 1 $ for the roots of the equation $ det(\lambda L + \lambda D + U) = 0 $  

To get accuracy = $ \varepsilon $, you should make iterations while $ \|x_{k+1} - x_{k}\| \le \frac{1-q}{q}\varepsilon $

In [13]:
def GaussSeidel(A, b, eps):
    checkDominance(A)
    n = b.size
    L = np.zeros((n,n))
    D = np.zeros((n,n))
    U = np.zeros((n,n))
    
    for i in range(n):
        for j in range(i):
            L[i,j] = A[i,j]
        D[i,i] = A[i,i]
        for j in range(i+1, n):
            U[i,j] = A[i,j]
    
    B = -la.inv(L + D).dot(U)
    f = la.inv(L + D).dot(b)
    x = np.zeros(n)
    x_new = np.zeros(n)
    q = la.norm(B, 1)
    if q >= 1:
        print("not converge")
        return x
    
    while True:
        x_new = B.dot(x) + f
        if la.norm(x_new - x, 2) <= (1-q)/q*eps:
            break
        x = x_new
        
    return x_new

# test
A1 = np.array([[ 1., -0.3,  0.2], 
               [0.3,  -1., -0.2], 
               [0.2, -0.3,   1.]])
b1 = np.array([1., -2.3, 2.6])
print("Embedded algorithm: ", la.solve(A1, b1))
print("Gauss-Seidel algorithm: ", GaussSeidel(A1, b1, 1e-5))

Embedded algorithm:  [1. 2. 3.]
diagonally dominant
Gauss-Seidel algorithm:  [1.00000065 2.00000025 2.99999994]


## 5.  Gradient descent method

As usual, we will solve a system $ Ax = b $  
Let $ Ф(x) $ be a quadratic functional: $ Ф(x) = (Ax, x) - 2(b, x) + c $,  
when $ (.,.) $ is a dot product of two vectors in a Hilbert space  
One can note that a gradient of this functional gives us our linear system: $ \nabla Ф = 2(Ax - b) $  
Then solution of our system coincide with a vector making $ Ф = min $  

Let's make an iterative process $ x_{k+1} = x_k - \tau_k r_k $,  
when $ r_k $ is known as residual: $ r_k = Ax_k - b $  
and $ \tau_k = \frac{(r_k, r_k)}{(Ar_k, r_k)} $  

Process can be ended, when $ \|r_k\| < \varepsilon $

In [14]:
def gradDesc(A, b, eps):
    n = b.size
    x = np.zeros(n)
    counter = 0
    while True:
        if counter > 1000:
            print("the method diverges")
            return x
        r = A.dot(x) - b
        if la.norm(r, 2) < eps:
            break
        tau = r.dot(r) / A.dot(r).dot(r)
        x = x - tau*r
        counter += 1
    return x

# test
A1 = np.array([[1., 1.],  
               [1., 2.]])
b1 = np.array([1., 1])
print("Embedded algorithm: ", la.solve(A1, b1))
print("Gradient descent algorithm: ", gradDesc(A1, b1, 1e-5))

Embedded algorithm:  [1. 0.]
Gradient descent algorithm:  [9.9999232e-01 5.1200000e-06]


## 6. Minimal residual method

This method is analogical to the previous.  
But now let's define $ \tau_k $ as $ \tau_k = \frac{(Ar_k, r_k)}{(Ar_k, Ar_k)} $  
This statement was taken from the condition of minimal norm of the residual: $ \|r_k\| = min $

In [15]:
def minRes(A, b, eps):
    n = b.size
    x = np.zeros(n)
    counter = 0
    while True:
        if counter > 1000:
            print("the method diverges")
            return x
        r = A.dot(x) - b
        if la.norm(r, 2) < eps:
            break
        tau = A.dot(r).dot(r) / A.dot(r).dot(A.dot(r))
        x = x - tau*r
        counter += 1
    return x

# test
A1 = np.array([[1., 1.],  
               [1., 2.]])
b1 = np.array([1., 1])
print("Embedded algorithm: ", la.solve(A1, b1))
print("Minimal residual algorithm: ", minRes(A1, b1, 1e-5))

Embedded algorithm:  [1. 0.]
Minimal residual algorithm:  [ 9.99997812e-01 -6.85215773e-17]
